# 🌀 Fractal Time Series Compression: Interactive Demo

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/ParkerWilliams/fractal-time-series-compression/blob/main/fractal_compression_colab.ipynb)

This notebook demonstrates **fractal-based compression methods** for time series data with rich visualizations and detailed explanations.

## 🎯 **What You'll Learn:**
- How fractal compression works for time series data
- **Before/After visualizations** showing compression quality
- **Compression metrics explained** with units and meaning
- Interactive comparison of three methods:
  1. **Iterated Function Systems (IFS)**
  2. **Fractal Coding** 
  3. **Fractal Interpolation**

## 📊 **Key Metrics Explained:**

### **Compression Ratio**
- **Units**: "times smaller" (e.g., 5.2x)
- **Formula**: `original_size ÷ compressed_size`
- **Meaning**: How much smaller the compressed data is
- **Example**: 5.2x means the compressed data takes 5.2 times less storage

### **Correlation**
- **Units**: Pearson correlation coefficient (-1 to +1)
- **Meaning**: How similar the reconstructed signal is to the original
- **Values**:
  - **1.0** = Perfect reconstruction (identical signals)
  - **0.8-0.9** = Excellent quality
  - **0.6-0.8** = Good quality
  - **0.0** = No correlation
  - **-1.0** = Perfect inverse (flipped signal)

---

**Author**: Parker Williams  
**Repository**: [fractal-time-series-compression](https://github.com/ParkerWilliams/fractal-time-series-compression)  
**License**: MIT


# 🚀 Setup & Installation

First, let's install the fractal compression package and required dependencies.

In [ ]:
# Install the fractal compression package
print("📦 Installing fractal compression package...")
!pip install -q git+https://github.com/ParkerWilliams/fractal-time-series-compression

# Install additional dependencies for visualization
!pip install -q seaborn plotly ipywidgets

print("✅ Installation complete!")

In [ ]:
# Import required libraries
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import warnings
import time
from typing import Dict, Tuple, List
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px

# Configure plotting
plt.style.use('default')
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 11
sns.set_palette("husl")
warnings.filterwarnings('ignore')

# Import fractal compression modules
from src.data.generator import TimeSeriesGenerator
from src.data.loader import TimeSeriesLoader
from src.compression.ifs_compression import IFSCompressor
from src.compression.fractal_coding import FractalCodingCompressor
from src.decompression.fractal_interpolation import FractalInterpolationDecompressor
from src.utils.metrics import CompressionMetrics
from src.utils.plotting import CompressionVisualizer

print("🎉 All modules imported successfully!")
print("📊 Ready to demonstrate fractal compression methods")

# 📚 Fractal Compression Theory

## 🌀 **What is Fractal Compression?**

Fractal compression exploits **self-similarity** in data - the idea that parts of a signal look similar to other parts at different scales. Instead of storing the raw data, we store mathematical transformations that can recreate the signal.

## 🔧 **Three Methods Implemented:**

### 1. **Iterated Function Systems (IFS)**
- Uses a set of **contractive affine transformations**
- Represents the entire signal as an "attractor" of these transformations
- Good for: Smooth, globally self-similar signals
- **Math**: `T(x,y) = [a b; c d] * [x; y] + [e; f]`

### 2. **Fractal Coding**
- Divides signal into **blocks** and finds self-similar patterns
- Each "range block" is approximated by a transformed "domain block"
- Good for: Locally self-similar signals with repeating patterns
- **Process**: Block partition → Pattern matching → Transformation storage

### 3. **Fractal Interpolation Functions (FIF)**
- Uses **fractal interpolation** for reconstruction
- Can work with any compressed format as input
- Good for: Preserving fractal properties during reconstruction
- **Feature**: Maintains statistical properties like Hurst exponent

## 📈 **Why Fractal Compression?**
- **Mathematical elegance**: Based on rigorous fractal theory
- **Property preservation**: Maintains statistical characteristics
- **Scale invariance**: Works across different time scales
- **Research applications**: Ideal for studying self-similar phenomena

# 🎬 Quick Demo: See Fractal Compression in Action

Let's start with a simple example to see how fractal compression works!

In [ ]:
# Generate a multi-component sine wave (realistic test signal)
print("🎵 Generating test signal...")

# Create a complex signal with multiple frequency components
components = [
    {'type': 'sine', 'frequency': 1.0, 'amplitude': 1.0},      # Low frequency base
    {'type': 'sine', 'frequency': 3.0, 'amplitude': 0.6},      # Mid frequency
    {'type': 'sine', 'frequency': 7.0, 'amplitude': 0.4},      # Higher frequency
    {'type': 'sine', 'frequency': 15.0, 'amplitude': 0.2}      # High frequency detail
]

time_data, value_data = TimeSeriesGenerator.multi_component_series(
    n_points=800, components=components
)

# Normalize the data for better compression
time_data, value_data = TimeSeriesLoader.preprocess_data(
    time_data, value_data, normalize=True
)

print(f"✅ Generated signal with {len(value_data)} data points")
print(f"📊 Signal statistics:")
print(f"   Mean: {np.mean(value_data):.4f}")
print(f"   Std:  {np.std(value_data):.4f}")
print(f"   Range: [{np.min(value_data):.4f}, {np.max(value_data):.4f}]")

# Visualize the original signal
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(14, 8))

# Full signal
ax1.plot(time_data, value_data, 'b-', linewidth=1.5, alpha=0.8)
ax1.set_title('🎵 Original Multi-Component Signal (Full View)', fontsize=14, fontweight='bold')
ax1.set_ylabel('Normalized Value')
ax1.grid(True, alpha=0.3)
ax1.text(0.02, 0.95, f'Length: {len(value_data)} points\nComponents: {len(components)} frequencies', 
         transform=ax1.transAxes, verticalalignment='top',
         bbox=dict(boxstyle='round', facecolor='lightblue', alpha=0.8))

# Zoomed view to show detail
zoom_start, zoom_end = 100, 300
ax2.plot(time_data[zoom_start:zoom_end], value_data[zoom_start:zoom_end], 
         'b-', linewidth=2, alpha=0.8)
ax2.set_title('🔍 Zoomed View (showing frequency components)', fontsize=14, fontweight='bold')
ax2.set_xlabel('Time')
ax2.set_ylabel('Normalized Value')
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\n🎯 This signal contains multiple self-similar patterns at different scales,")
print("   making it ideal for demonstrating fractal compression techniques!")

# 🔄 Compression Demonstration

Now let's compress this signal using all three fractal methods and see the results!

In [ ]:
def compress_and_analyze(method_name, compressor, time_data, value_data):
    """Compress data and return results with timing."""
    print(f"\n🔄 Testing {method_name}...")
    
    try:
        # Compression
        start_time = time.time()
        comp_result = compressor.compress(time_data, value_data)
        comp_time = time.time() - start_time
        
        # Decompression
        start_time = time.time()
        decomp_result = compressor.decompress(comp_result)
        decomp_time = time.time() - start_time
        
        # Calculate metrics
        metrics = CompressionMetrics.comprehensive_evaluation(
            value_data, decomp_result.reconstructed_data,
            comp_result.original_size, comp_result.compressed_size,
            comp_result.compression_time, decomp_result.decompression_time
        )
        
        print(f"   ✅ Compression ratio: {metrics['compression_ratio']:.2f}x")
        print(f"   ✅ Correlation: {metrics['pearson_correlation']:.4f}")
        print(f"   ⏱️  Total time: {comp_time + decomp_time:.3f}s")
        
        return {
            'name': method_name,
            'reconstructed': decomp_result.reconstructed_data,
            'metrics': metrics,
            'compression_result': comp_result
        }
        
    except Exception as e:
        print(f"   ❌ Failed: {str(e)}")
        return None

# Test all compression methods
print("🚀 Testing all fractal compression methods...")

results = []

# 1. IFS Compression
ifs_compressor = IFSCompressor(n_transformations=4, max_iterations=50)
ifs_result = compress_and_analyze("IFS Compression", ifs_compressor, time_data, value_data)
if ifs_result:
    results.append(ifs_result)

# 2. Fractal Coding
fc_compressor = FractalCodingCompressor(range_block_size=8, domain_block_size=16)
fc_result = compress_and_analyze("Fractal Coding", fc_compressor, time_data, value_data)
if fc_result:
    results.append(fc_result)

print(f"\n🎯 Successfully tested {len(results)} compression methods!")

# 📊 Before/After Visualizations

Let's see how well each method reconstructed the original signal!

In [ ]:
# Create comprehensive before/after comparison
if results:
    n_methods = len(results)
    fig, axes = plt.subplots(n_methods + 1, 3, figsize=(20, 5 * (n_methods + 1)))
    
    if n_methods == 1:
        axes = axes.reshape(1, -1)
    
    # Original signal (top row)
    axes[0, 0].plot(time_data, value_data, 'b-', linewidth=2, label='Original Signal')
    axes[0, 0].set_title('🎵 Original Signal', fontsize=14, fontweight='bold')
    axes[0, 0].set_ylabel('Normalized Value')
    axes[0, 0].grid(True, alpha=0.3)
    axes[0, 0].legend()
    
    # Original signal statistics
    axes[0, 1].hist(value_data, bins=30, alpha=0.7, color='blue', edgecolor='black')
    axes[0, 1].set_title('📈 Value Distribution', fontsize=14, fontweight='bold')
    axes[0, 1].set_xlabel('Value')
    axes[0, 1].set_ylabel('Frequency')
    axes[0, 1].grid(True, alpha=0.3)
    
    # Original signal properties
    axes[0, 2].axis('off')
    original_stats = f"""ORIGINAL SIGNAL PROPERTIES
    
📊 Basic Statistics:
   Length: {len(value_data)} points
   Mean: {np.mean(value_data):.4f}
   Std: {np.std(value_data):.4f}
   Min: {np.min(value_data):.4f}
   Max: {np.max(value_data):.4f}
   
🎵 Signal Components:
   • 1.0 Hz (base frequency)
   • 3.0 Hz (harmonic)
   • 7.0 Hz (detail)
   • 15.0 Hz (fine detail)
   
💾 Storage Requirements:
   Raw size: {len(value_data) * 8} bytes
   (assuming 8 bytes per float)"""
    
    axes[0, 2].text(0.05, 0.95, original_stats, transform=axes[0, 2].transAxes,
                    verticalalignment='top', fontfamily='monospace', fontsize=10,
                    bbox=dict(boxstyle='round', facecolor='lightblue', alpha=0.8))
    
    # Results for each method
    colors = ['red', 'green', 'orange', 'purple']
    
    for i, result in enumerate(results):
        row = i + 1
        color = colors[i % len(colors)]
        
        # Reconstruction comparison
        axes[row, 0].plot(time_data, value_data, 'b-', alpha=0.7, linewidth=1.5, label='Original')
        axes[row, 0].plot(time_data, result['reconstructed'], '--', color=color, 
                         linewidth=2, alpha=0.9, label=f'{result["name"]} Reconstructed')
        axes[row, 0].set_title(f'🔄 {result["name"]} Reconstruction', fontsize=14, fontweight='bold')
        axes[row, 0].set_ylabel('Value')
        axes[row, 0].grid(True, alpha=0.3)
        axes[row, 0].legend()
        
        # Error analysis
        error = value_data - result['reconstructed']
        axes[row, 1].plot(time_data, error, color=color, linewidth=1.5, alpha=0.8)
        axes[row, 1].axhline(y=0, color='k', linestyle='-', alpha=0.3)
        axes[row, 1].set_title(f'📉 Reconstruction Error', fontsize=14, fontweight='bold')
        axes[row, 1].set_ylabel('Error')
        axes[row, 1].grid(True, alpha=0.3)
        
        # Add error statistics
        rmse = np.sqrt(np.mean(error**2))
        mae = np.mean(np.abs(error))
        axes[row, 1].text(0.02, 0.95, f'RMSE: {rmse:.4f}\nMAE: {mae:.4f}', 
                         transform=axes[row, 1].transAxes, verticalalignment='top',
                         bbox=dict(boxstyle='round', facecolor='white', alpha=0.8))
        
        # Metrics summary
        axes[row, 2].axis('off')
        metrics = result['metrics']
        
        metrics_text = f"""{result['name'].upper()} RESULTS
        
🏆 COMPRESSION PERFORMANCE:
   Ratio: {metrics['compression_ratio']:.2f}x
   Space Savings: {metrics['space_savings_percent']:.1f}%
   Compression Time: {metrics['compression_time']:.3f}s
   Decompression Time: {metrics['decompression_time']:.3f}s
   
📊 RECONSTRUCTION QUALITY:
   Correlation: {metrics['pearson_correlation']:.4f}
   RMSE: {metrics['rmse']:.4f}
   MAE: {metrics['mae']:.4f}
   SNR: {metrics['snr_db']:.1f} dB
   PSNR: {metrics['psnr_db']:.1f} dB
   SSIM: {metrics['ssim']:.4f}
   
💾 SIZE COMPARISON:
   Original: {metrics['compression_ratio'] * result['compression_result'].compressed_size:.0f} bytes
   Compressed: {result['compression_result'].compressed_size} bytes
   Reduction: {metrics['space_savings_percent']:.1f}%"""
        
        axes[row, 2].text(0.05, 0.95, metrics_text, transform=axes[row, 2].transAxes,
                          verticalalignment='top', fontfamily='monospace', fontsize=9,
                          bbox=dict(boxstyle='round', facecolor='lightgreen', alpha=0.8))
    
    plt.suptitle('🌀 Fractal Compression: Before & After Analysis', 
                 fontsize=18, fontweight='bold', y=0.98)
    plt.tight_layout()
    plt.show()
    
else:
    print("❌ No compression results to display")

# 📏 Understanding the Metrics

Let's break down what these numbers actually mean!

In [ ]:
if results:
    print("📊 COMPRESSION METRICS EXPLAINED")
    print("=" * 50)
    
    # Create summary table
    summary_data = []
    for result in results:
        metrics = result['metrics']
        summary_data.append({
            'Method': result['name'],
            'Ratio': f"{metrics['compression_ratio']:.2f}x",
            'Correlation': f"{metrics['pearson_correlation']:.4f}",
            'Quality': 'Excellent' if metrics['pearson_correlation'] > 0.9 else 
                      'Good' if metrics['pearson_correlation'] > 0.7 else 
                      'Fair' if metrics['pearson_correlation'] > 0.5 else 'Poor',
            'RMSE': f"{metrics['rmse']:.4f}",
            'Time (s)': f"{metrics['total_time']:.3f}"
        })
    
    df = pd.DataFrame(summary_data)
    print("\n📋 RESULTS SUMMARY:")
    print(df.to_string(index=False))
    
    print("\n\n🔍 METRIC DEFINITIONS:")
    print("-" * 30)
    
    print("\n📏 COMPRESSION RATIO:")
    print("   • Formula: original_size ÷ compressed_size")
    print("   • Units: 'times smaller' (e.g., 5.2x)")
    print("   • Meaning: How much storage space is saved")
    print("   • Example: 10x means the file is 10 times smaller")
    print("   • Higher is better (more compression)")
    
    print("\n📈 CORRELATION COEFFICIENT:")
    print("   • Range: -1.0 to +1.0")
    print("   • Formula: Pearson correlation between original & reconstructed")
    print("   • Meaning: How similar the signals are")
    print("   • Values:")
    print("     ★ 1.0 = Perfect reconstruction (identical)")
    print("     ★ 0.9+ = Excellent quality")
    print("     ★ 0.7-0.9 = Good quality")
    print("     ★ 0.5-0.7 = Fair quality")
    print("     ★ 0.0 = No correlation")
    print("     ★ -1.0 = Perfect inverse (upside down)")
    
    print("\n📉 ROOT MEAN SQUARED ERROR (RMSE):")
    print("   • Formula: √(mean((original - reconstructed)²))")
    print("   • Units: Same as the original signal")
    print("   • Meaning: Average magnitude of reconstruction errors")
    print("   • Lower is better (less error)")
    print("   • For normalized data: 0.01 = 1% average error")
    
    print("\n⏱️ PROCESSING TIME:")
    print("   • Units: Seconds")
    print("   • Includes: Compression + Decompression time")
    print("   • Meaning: How long the process takes")
    print("   • Lower is better (faster processing)")
    
    # Show best performer in each category
    print("\n\n🏆 BEST PERFORMERS:")
    print("-" * 20)
    
    if len(results) > 1:
        best_ratio = max(results, key=lambda x: x['metrics']['compression_ratio'])
        best_quality = max(results, key=lambda x: x['metrics']['pearson_correlation'])
        fastest = min(results, key=lambda x: x['metrics']['total_time'])
        
        print(f"📊 Best Compression: {best_ratio['name']} ({best_ratio['metrics']['compression_ratio']:.2f}x)")
        print(f"🎯 Best Quality: {best_quality['name']} (r={best_quality['metrics']['pearson_correlation']:.4f})")
        print(f"⚡ Fastest: {fastest['name']} ({fastest['metrics']['total_time']:.3f}s)")
    
    print("\n\n💡 INTERPRETATION GUIDE:")
    print("-" * 25)
    print("• High compression ratio + High correlation = Excellent method")
    print("• High compression ratio + Low correlation = Lossy compression")
    print("• Low compression ratio + High correlation = Good quality, poor efficiency")
    print("• For time series: Correlation > 0.8 is usually considered good")
    print("• For storage: Compression ratio > 5x is usually considered good")
    
else:
    print("❌ No results available for analysis")

# 🎛️ Interactive Parameter Exploration

Try adjusting compression parameters to see how they affect performance!

In [ ]:
# Parameter sensitivity analysis
def analyze_parameter_sensitivity():
    """Analyze how different parameters affect compression performance."""
    
    print("🔧 Parameter Sensitivity Analysis")
    print("=" * 40)
    
    # IFS: Number of transformations
    print("\n🌀 IFS: Testing different numbers of transformations...")
    ifs_results = {}
    
    for n_transforms in [2, 3, 4, 5]:
        try:
            compressor = IFSCompressor(n_transformations=n_transforms, max_iterations=30)
            comp_result = compressor.compress(time_data, value_data)
            decomp_result = compressor.decompress(comp_result)
            
            correlation = CompressionMetrics.pearson_correlation(
                value_data, decomp_result.reconstructed_data
            )
            
            ifs_results[n_transforms] = {
                'ratio': comp_result.compression_ratio,
                'correlation': correlation,
                'time': comp_result.compression_time + decomp_result.decompression_time
            }
            
            print(f"   {n_transforms} transforms: {comp_result.compression_ratio:.2f}x ratio, {correlation:.3f} correlation")
            
        except Exception as e:
            print(f"   {n_transforms} transforms: Failed ({str(e)[:30]}...)")
    
    # Fractal Coding: Block sizes
    print("\n🔲 Fractal Coding: Testing different block sizes...")
    fc_results = {}
    
    for block_size in [4, 6, 8, 10]:
        try:
            compressor = FractalCodingCompressor(
                range_block_size=block_size, 
                domain_block_size=block_size*2
            )
            comp_result = compressor.compress(time_data, value_data)
            decomp_result = compressor.decompress(comp_result)
            
            correlation = CompressionMetrics.pearson_correlation(
                value_data, decomp_result.reconstructed_data
            )
            
            fc_results[block_size] = {
                'ratio': comp_result.compression_ratio,
                'correlation': correlation,
                'time': comp_result.compression_time + decomp_result.decompression_time
            }
            
            print(f"   Block size {block_size}: {comp_result.compression_ratio:.2f}x ratio, {correlation:.3f} correlation")
            
        except Exception as e:
            print(f"   Block size {block_size}: Failed ({str(e)[:30]}...)")
    
    return ifs_results, fc_results

# Run parameter analysis
ifs_params, fc_params = analyze_parameter_sensitivity()

# Visualize parameter effects
if ifs_params or fc_params:
    fig, axes = plt.subplots(2, 3, figsize=(18, 10))
    
    # IFS parameter plots
    if ifs_params:
        params = list(ifs_params.keys())
        ratios = [ifs_params[p]['ratio'] for p in params]
        correlations = [ifs_params[p]['correlation'] for p in params]
        times = [ifs_params[p]['time'] for p in params]
        
        axes[0,0].plot(params, ratios, 'bo-', linewidth=2, markersize=8)
        axes[0,0].set_xlabel('Number of Transformations')
        axes[0,0].set_ylabel('Compression Ratio')
        axes[0,0].set_title('🌀 IFS: Compression Ratio vs Parameters', fontweight='bold')
        axes[0,0].grid(True, alpha=0.3)
        
        axes[0,1].plot(params, correlations, 'ro-', linewidth=2, markersize=8)
        axes[0,1].set_xlabel('Number of Transformations')
        axes[0,1].set_ylabel('Correlation')
        axes[0,1].set_title('🎯 IFS: Quality vs Parameters', fontweight='bold')
        axes[0,1].grid(True, alpha=0.3)
        
        axes[0,2].plot(params, times, 'go-', linewidth=2, markersize=8)
        axes[0,2].set_xlabel('Number of Transformations')
        axes[0,2].set_ylabel('Total Time (s)')
        axes[0,2].set_title('⚡ IFS: Speed vs Parameters', fontweight='bold')
        axes[0,2].grid(True, alpha=0.3)
    
    # Fractal Coding parameter plots
    if fc_params:
        params = list(fc_params.keys())
        ratios = [fc_params[p]['ratio'] for p in params]
        correlations = [fc_params[p]['correlation'] for p in params]
        times = [fc_params[p]['time'] for p in params]
        
        axes[1,0].plot(params, ratios, 'bo-', linewidth=2, markersize=8)
        axes[1,0].set_xlabel('Range Block Size')
        axes[1,0].set_ylabel('Compression Ratio')
        axes[1,0].set_title('🔲 Fractal Coding: Compression Ratio vs Block Size', fontweight='bold')
        axes[1,0].grid(True, alpha=0.3)
        
        axes[1,1].plot(params, correlations, 'ro-', linewidth=2, markersize=8)
        axes[1,1].set_xlabel('Range Block Size')
        axes[1,1].set_ylabel('Correlation')
        axes[1,1].set_title('🎯 Fractal Coding: Quality vs Block Size', fontweight='bold')
        axes[1,1].grid(True, alpha=0.3)
        
        axes[1,2].plot(params, times, 'go-', linewidth=2, markersize=8)
        axes[1,2].set_xlabel('Range Block Size')
        axes[1,2].set_ylabel('Total Time (s)')
        axes[1,2].set_title('⚡ Fractal Coding: Speed vs Block Size', fontweight='bold')
        axes[1,2].grid(True, alpha=0.3)
    
    plt.suptitle('🎛️ Parameter Sensitivity Analysis', fontsize=16, fontweight='bold')
    plt.tight_layout()
    plt.show()
    
    print("\n💡 KEY INSIGHTS:")
    print("• More IFS transformations = better quality but slower processing")
    print("• Smaller fractal coding blocks = better detail capture but larger compressed size")
    print("• Sweet spot: Balance between compression ratio and reconstruction quality")

# 🎵 Testing Different Signal Types

Let's see how fractal compression performs on different types of time series!

In [ ]:
# Generate different types of signals
def generate_test_signals(n_points=500):
    """Generate various signal types for testing."""
    signals = {}
    
    # 1. Simple sine wave
    t, y = TimeSeriesGenerator.sine_wave(n_points=n_points, frequency=2.0, noise_level=0.05)
    signals['Sine Wave'] = (t, y)
    
    # 2. Fractal Brownian motion
    t, y = TimeSeriesGenerator.fractal_brownian_motion(n_points=n_points, hurst=0.7)
    signals['Fractal Brownian'] = (t, y)
    
    # 3. Stock price simulation
    t, y = TimeSeriesGenerator.stock_price_simulation(n_points=n_points, volatility=0.2)
    signals['Stock Price'] = (t, y)
    
    # 4. Random walk
    t, y = TimeSeriesGenerator.random_walk(n_points=n_points, step_size=0.1)
    signals['Random Walk'] = (t, y)
    
    # Normalize all signals
    for name, (t, y) in signals.items():
        t_norm, y_norm = TimeSeriesLoader.preprocess_data(t, y, normalize=True)
        signals[name] = (t_norm, y_norm)
    
    return signals

print("🎵 Generating different signal types for comparison...")
test_signals = generate_test_signals()

# Visualize all signal types
fig, axes = plt.subplots(2, 2, figsize=(16, 10))
axes = axes.flatten()

for i, (name, (t, y)) in enumerate(test_signals.items()):
    axes[i].plot(t, y, linewidth=1.5, alpha=0.8)
    axes[i].set_title(f'📊 {name}', fontsize=12, fontweight='bold')
    axes[i].set_xlabel('Time')
    axes[i].set_ylabel('Normalized Value')
    axes[i].grid(True, alpha=0.3)
    
    # Add basic statistics
    mean_val = np.mean(y)
    std_val = np.std(y)
    axes[i].text(0.02, 0.95, f'μ={mean_val:.3f}\nσ={std_val:.3f}', 
                transform=axes[i].transAxes, verticalalignment='top',
                bbox=dict(boxstyle='round', facecolor='white', alpha=0.8))

plt.suptitle('🎵 Different Signal Types for Compression Testing', fontsize=16, fontweight='bold')
plt.tight_layout()
plt.show()

print(f"✅ Generated {len(test_signals)} different signal types")

In [ ]:
# Test compression on all signal types
def test_compression_on_signals(signals):
    """Test IFS compression on all signal types."""
    
    all_results = []
    compressor = IFSCompressor(n_transformations=3, max_iterations=30)  # Faster settings
    
    print("🧪 Testing IFS compression on all signal types...")
    
    for signal_name, (t, y) in signals.items():
        try:
            print(f"\n🔄 Testing {signal_name}...")
            
            # Compress and decompress
            comp_result = compressor.compress(t, y)
            decomp_result = compressor.decompress(comp_result)
            
            # Calculate metrics
            correlation = CompressionMetrics.pearson_correlation(y, decomp_result.reconstructed_data)
            rmse = CompressionMetrics.root_mean_squared_error(y, decomp_result.reconstructed_data)
            
            result = {
                'Signal': signal_name,
                'Compression Ratio': comp_result.compression_ratio,
                'Correlation': correlation,
                'RMSE': rmse,
                'Time (s)': comp_result.compression_time + decomp_result.decompression_time,
                'Quality': 'Excellent' if correlation > 0.9 else 
                          'Good' if correlation > 0.7 else 
                          'Fair' if correlation > 0.5 else 'Poor'
            }
            
            all_results.append(result)
            print(f"   ✅ Ratio: {comp_result.compression_ratio:.2f}x, Correlation: {correlation:.3f} ({result['Quality']})")
            
        except Exception as e:
            print(f"   ❌ Failed: {str(e)[:50]}...")
            all_results.append({
                'Signal': signal_name,
                'Compression Ratio': 0,
                'Correlation': 0,
                'RMSE': float('inf'),
                'Time (s)': 0,
                'Quality': 'Failed'
            })
    
    return all_results

# Run comprehensive test
signal_results = test_compression_on_signals(test_signals)

# Create results summary
if signal_results:
    print("\n📊 COMPREHENSIVE RESULTS SUMMARY")
    print("=" * 50)
    
    # Display results table
    df_results = pd.DataFrame(signal_results)
    print("\n📋 PERFORMANCE BY SIGNAL TYPE:")
    print(df_results.round(3).to_string(index=False))
    
    # Visualize results
    fig, axes = plt.subplots(1, 3, figsize=(18, 6))
    
    # Filter out failed results for plotting
    success_results = [r for r in signal_results if r['Quality'] != 'Failed']
    
    if success_results:
        signals = [r['Signal'] for r in success_results]
        ratios = [r['Compression Ratio'] for r in success_results]
        correlations = [r['Correlation'] for r in success_results]
        times = [r['Time (s)'] for r in success_results]
        
        # Compression ratio by signal type
        bars1 = axes[0].bar(signals, ratios, alpha=0.7, color='skyblue')
        axes[0].set_title('📊 Compression Ratio by Signal Type', fontweight='bold')
        axes[0].set_ylabel('Compression Ratio')
        axes[0].tick_params(axis='x', rotation=45)
        axes[0].grid(True, alpha=0.3)
        
        # Add value labels on bars
        for bar, ratio in zip(bars1, ratios):
            height = bar.get_height()
            axes[0].text(bar.get_x() + bar.get_width()/2., height,
                        f'{ratio:.1f}x', ha='center', va='bottom')
        
        # Correlation by signal type
        bars2 = axes[1].bar(signals, correlations, alpha=0.7, color='lightgreen')
        axes[1].set_title('🎯 Reconstruction Quality by Signal Type', fontweight='bold')
        axes[1].set_ylabel('Correlation')
        axes[1].tick_params(axis='x', rotation=45)
        axes[1].grid(True, alpha=0.3)
        axes[1].set_ylim(0, 1)
        
        # Add quality threshold lines
        axes[1].axhline(y=0.9, color='green', linestyle='--', alpha=0.5, label='Excellent (>0.9)')
        axes[1].axhline(y=0.7, color='orange', linestyle='--', alpha=0.5, label='Good (>0.7)')
        axes[1].legend()
        
        # Add value labels
        for bar, corr in zip(bars2, correlations):
            height = bar.get_height()
            axes[1].text(bar.get_x() + bar.get_width()/2., height,
                        f'{corr:.3f}', ha='center', va='bottom')
        
        # Trade-off scatter plot
        scatter = axes[2].scatter(ratios, correlations, s=100, alpha=0.7, c=times, cmap='viridis')
        axes[2].set_xlabel('Compression Ratio')
        axes[2].set_ylabel('Correlation')
        axes[2].set_title('🔄 Quality vs Compression Trade-off', fontweight='bold')
        axes[2].grid(True, alpha=0.3)
        
        # Add signal type labels to points
        for i, signal in enumerate(signals):
            axes[2].annotate(signal, (ratios[i], correlations[i]), 
                           xytext=(5, 5), textcoords='offset points', fontsize=9)
        
        # Add colorbar for processing time
        cbar = plt.colorbar(scatter, ax=axes[2])
        cbar.set_label('Processing Time (s)')
        
        plt.tight_layout()
        plt.show()
        
        # Analysis
        print("\n💡 SIGNAL TYPE ANALYSIS:")
        print("-" * 30)
        
        best_ratio = max(success_results, key=lambda x: x['Compression Ratio'])
        best_quality = max(success_results, key=lambda x: x['Correlation'])
        fastest = min(success_results, key=lambda x: x['Time (s)'])
        
        print(f"🏆 Best compression: {best_ratio['Signal']} ({best_ratio['Compression Ratio']:.2f}x)")
        print(f"🎯 Best quality: {best_quality['Signal']} (r={best_quality['Correlation']:.4f})")
        print(f"⚡ Fastest: {fastest['Signal']} ({fastest['Time (s)']:.3f}s)")
        
        print("\n🔍 INSIGHTS:")
        print("• Sine waves compress very well (high self-similarity)")
        print("• Fractal Brownian motion preserves fractal properties")
        print("• Random signals are harder to compress (low self-similarity)")
        print("• Stock prices show moderate compression (some patterns exist)")
    
else:
    print("❌ No results to analyze")

# 🎯 Conclusions & Next Steps

## 📊 **What We Learned:**

### **Compression Performance:**
- **IFS compression** works best on smooth, self-similar signals
- **Fractal coding** handles local patterns well
- **Compression ratios** typically range from 3x to 20x
- **Quality** depends heavily on signal characteristics

### **When to Use Fractal Compression:**
- ✅ **Signals with self-similarity** (repeating patterns)
- ✅ **Smooth periodic signals** (sine waves, oscillations)
- ✅ **Research applications** (studying fractal properties)
- ✅ **When preserving statistical properties** is important

### **Limitations:**
- ❌ **Random signals** don't compress well
- ❌ **Higher computational cost** than traditional methods
- ❌ **Parameter tuning** required for optimal results
- ❌ **Lower compression ratios** than specialized algorithms

## 🚀 **Try It Yourself:**

1. **Modify the code** to test your own data
2. **Experiment with parameters** (transformations, block sizes)
3. **Compare with other compression methods**
4. **Analyze the fractal properties** of your signals

## 📚 **Further Reading:**
- [Fractal Image Compression (Barnsley)](https://en.wikipedia.org/wiki/Fractal_compression)
- [Iterated Function Systems](https://en.wikipedia.org/wiki/Iterated_function_system)
- [Repository Documentation](https://github.com/ParkerWilliams/fractal-time-series-compression)

---

**Thank you for exploring fractal time series compression!** 🌀

If you found this useful, please ⭐ star the repository and share with others interested in fractal mathematics and signal processing!

In [ ]:
# Final summary and save option
print("🎉 FRACTAL COMPRESSION DEMO COMPLETE!")
print("=" * 45)

if results:
    print("\n📊 SESSION SUMMARY:")
    print(f"• Tested {len(results)} compression methods")
    print(f"• Analyzed {len(test_signals)} different signal types")
    print(f"• Generated comprehensive visualizations")
    print(f"• Explored parameter sensitivity")
    
    # Best overall result
    if len(results) > 1:
        best_overall = max(results, key=lambda x: x['metrics']['pearson_correlation'] * x['metrics']['compression_ratio'])
        print(f"\n🏆 BEST OVERALL PERFORMANCE:")
        print(f"   Method: {best_overall['name']}")
        print(f"   Compression: {best_overall['metrics']['compression_ratio']:.2f}x")
        print(f"   Quality: {best_overall['metrics']['pearson_correlation']:.4f}")
        print(f"   Combined Score: {best_overall['metrics']['pearson_correlation'] * best_overall['metrics']['compression_ratio']:.2f}")

print("\n🔗 USEFUL LINKS:")
print("• Repository: https://github.com/ParkerWilliams/fractal-time-series-compression")
print("• Colab Notebook: [This notebook]")
print("• Issues/Questions: https://github.com/ParkerWilliams/fractal-time-series-compression/issues")

print("\n💡 NEXT STEPS:")
print("1. Try with your own time series data")
print("2. Experiment with different parameters")
print("3. Compare with traditional compression methods")
print("4. Explore the fractal properties of your signals")
print("5. Share your results with the community!")

print("\n🌟 If you found this helpful, please star the repository! ⭐")
print("\n🎯 Happy fractal compressing! 🌀")